In [1]:
!pip install transformers
!pip install sentence-transformers
!pip3 install truecase


## for data
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold
from sklearn.metrics.pairwise import cosine_similarity

## for processing
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
import string

## for w2v
import gensim
import gensim.downloader as gensim_api

## for bert
import transformers

from sentence_transformers import SentenceTransformer


import spacy
from spacy import displacy

import truecase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
Question = input("What do you want to know? \n")

What do you want to know? 
What are some places for families to visit


# Question Classification

In [3]:

## Create Dictionary {category:[keywords]}
dic_clusters = {}
dic_clusters["PLACE"] = ['what', 'culture', 'this', 'place', 'myth', 'fact', 'of', 'which', 'just', 'there', 'historical', 'in', 'life', 'modern', 'history', 'the', 'where', 'part', 'tradition', 'town', 'one', 'village', 'century', 'time', 'ancient', 'is', 'it', 'same', '.']
dic_clusters["BEACH"] = ['quiet', 'inland', 'sea', 'fish', 'water', 'surf', 'along', 'lagoon', 'pond', 'mud', 'swimming', 'waters', 'boat', 'coastal', 'shallow', 'shores', 'shoreline', 'beach', 'swim', 'shore', 'boats', 'washed', 'river', 'surface', 'seas', 'beaches', 'fishing', 'lake', 'coral', 'sand', 'ocean']
dic_clusters["ACTIVITY"] = ['walking', 'concert', 'riding', 'this', 'ride', 'rides', 'trip', 'coming', 'cycling', 'activities', 'events', 'bike', '.', 'event', 'day', 'festival', 'weekend', 'year', 'traveling', 'part', 'activity', 'festivals', 'trips', 'summer', 'tours', 'tour', 'same', 'during', 'touring']
dic_clusters["TOURIST_ATTRACTION"] = ['monument', 'church', 'famous', 'historical', 'attractions', 'temple', 'cathedral', 'monastery', 'historic', 'near', 'site', 'built', 'volcano', 'ruins', 'castle', 'museum', 'situated', 'archaeological', 'town', 'village', 'nearby', 'century', 'palace', 'attraction', 'medieval', 'heritage', 'ancient', 'located', 'chapel', 'park', 'preserved']
dic_clusters["LOCAL_FOOD_DRINK"] = ['restaurants', 'dish', 'snacks', 'foods', 'drink', 'drinks', 'meat', 'bread', 'dessert', 'beer', 'cooked', 'traditional', 'cooking', 'eating', 'taste', 'meals', 'fruits', 'cuisine', 'food', 'eaten', 'ingredients', 'coffee', 'fruit', 'vegetable', 'seafood', 'chicken', 'eat', 'meal', 'dishes', 'vegetables', 'wine', 'ate']
dic_clusters["NOTABLE_PERSON"] = ['woman', 'father', 'named', 'brother', 'married', 'names', 'notable', 'famous', 'death', 'whose', 'person', 'whom', 'only', 'known', 'as', 'lived', 'who', 'although', 'name', 'one', 'son', 'born', 'mother', '—', 'daughter', 'may', 'man', 'died', 'same', ',']
dic_clusters["CLIMATE"] = ['rainfall', 'dry', 'winter', 'precipitation', 'weather', 'month', 'change', 'storms', 'rains', 'temperatures', 'warmer', 'temperature', 'cold', 'warming', 'wind', 'humidity', 'warm', 'humid', 'rain', 'day', 'winds', 'snow', 'forecast', 'expected', 'due', 'climate', 'days', 'conditions', 'cooler', 'rainy']
dic_clusters["HEALTHCARE"] = ['nurse', 'emergency', 'ambulance', 'doctor', 'physicians', 'veterinary', 'hospitals', 'medicine', 'dental', 'nurses', 'nursing', 'dr.', 'pediatric', 'medical', 'doctors', 'patient', 'services', 'treatment', 'healthcare', 'surgery', 'patients', 'outpatient', 'health', 'hospital', 'care', 'physician', 'psychiatric', 'surgical', 'clinic', 'clinical']
dic_clusters["INFORMATION"] = ['information', 'means', 'provides', 'useful', 'can', 'without', 'use', 'provided', 'certain', 'data', 'these', 'access', 'knowledge', 'you', 'using', 'allows', 'example', 'providing', 'instance', 'need', 'available', 'driving', 'atm', 'rent', 'provide', 'specific', 'money', 'required', 'rather', 'how', 'any']

## print some
for k,v in dic_clusters.items():
    print(k, ": ", v[0:len(v)], "...", len(v))


PLACE :  ['what', 'culture', 'this', 'place', 'myth', 'fact', 'of', 'which', 'just', 'there', 'historical', 'in', 'life', 'modern', 'history', 'the', 'where', 'part', 'tradition', 'town', 'one', 'village', 'century', 'time', 'ancient', 'is', 'it', 'same', '.'] ... 29
BEACH :  ['quiet', 'inland', 'sea', 'fish', 'water', 'surf', 'along', 'lagoon', 'pond', 'mud', 'swimming', 'waters', 'boat', 'coastal', 'shallow', 'shores', 'shoreline', 'beach', 'swim', 'shore', 'boats', 'washed', 'river', 'surface', 'seas', 'beaches', 'fishing', 'lake', 'coral', 'sand', 'ocean'] ... 31
ACTIVITY :  ['walking', 'concert', 'riding', 'this', 'ride', 'rides', 'trip', 'coming', 'cycling', 'activities', 'events', 'bike', '.', 'event', 'day', 'festival', 'weekend', 'year', 'traveling', 'part', 'activity', 'festivals', 'trips', 'summer', 'tours', 'tour', 'same', 'during', 'touring'] ... 29
TOURIST_ATTRACTION :  ['monument', 'church', 'famous', 'historical', 'attractions', 'temple', 'cathedral', 'monastery', 'hist

In [4]:
#Removing Punctuation, Null entries and stopwords

def Cleaning_1_1(dic_clusters):

  lst_stopwords = nltk.corpus.stopwords.words("english")

  for k,v in dic_clusters.items():
    for word in v:
      if len(word)<=2:
        v.remove(word)

    for i in lst_stopwords:
      if i in v:
        v.remove(i)

    new_list = v
    dic_clusters.update({k: list(filter(None, new_list))})

  return dic_clusters

In [5]:
#Removing duplicates between the dictionaries

def Cleaning_1_2(dic_clusters):
  seen = set()
  repeated = set()
  new_list=[]

  for k,v in dic_clusters.items():
    for i in set(v):
      if i in seen:
        repeated.add(i)
      else:
        seen.add(i) 
    
  for i in repeated:
    x=0
    for k,v in dic_clusters.items():
      new_list2 = v
      if i in v:
        x=x+1
        if x>1:
          new_list2.remove(i)
     
      dic_clusters.update({k: new_list2})


In [6]:
Cleaning_1_1(dic_clusters)
Cleaning_1_2(dic_clusters)

for k,v in dic_clusters.items():
    print(k, ": ", v[0:len(v)], "...", len(v))

PLACE :  ['culture', 'place', 'myth', 'fact', 'historical', 'life', 'modern', 'history', 'part', 'tradition', 'town', 'one', 'village', 'century', 'time', 'ancient'] ... 16
BEACH :  ['quiet', 'inland', 'sea', 'fish', 'water', 'surf', 'along', 'lagoon', 'pond', 'mud', 'swimming', 'waters', 'boat', 'coastal', 'shallow', 'shores', 'shoreline', 'beach', 'swim', 'shore', 'boats', 'washed', 'river', 'surface', 'seas', 'beaches', 'fishing', 'lake', 'coral', 'sand', 'ocean'] ... 31
ACTIVITY :  ['walking', 'concert', 'riding', 'ride', 'rides', 'trip', 'coming', 'cycling', 'activities', 'events', 'bike', 'event', 'day', 'festival', 'weekend', 'year', 'traveling', 'activity', 'festivals', 'trips', 'summer', 'tours', 'tour', 'touring'] ... 24
TOURIST_ATTRACTION :  ['monument', 'church', 'famous', 'attractions', 'temple', 'cathedral', 'monastery', 'historic', 'near', 'site', 'built', 'volcano', 'ruins', 'castle', 'museum', 'situated', 'archaeological', 'nearby', 'palace', 'attraction', 'medieval', 

In [7]:
#preprocessing and cleaning method

'''
Preprocess a string.
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_stemm: bool - whether stemming is to be applied
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()    
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [8]:
lst_stopwords = nltk.corpus.stopwords.words("english")
#print(lst_stopwords)

Clean_q = utils_preprocess_text(Question, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords)

In [9]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
nlp = transformers.TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [10]:
## function to apply
def utils_bert_embedding(txt, tokenizer, nlp):
    idx = tokenizer.encode(txt)
    idx = np.array(idx)[None,:]  
    embedding = nlp(idx)
    X = np.array(embedding[0][0][1:-1])
    return X


lst_mean_vecs = utils_bert_embedding(Clean_q, tokenizer, nlp).mean(0) 

X = np.array(lst_mean_vecs)

In [11]:
dic_y = {k:utils_bert_embedding(v, tokenizer, nlp).mean(0) for k,v
         in dic_clusters.items()}

In [12]:
#--- Model Algorithm ---#
## compute cosine similarities
similarities_q = np.array(
            [metrics.pairwise.cosine_similarity(X.reshape(1, -1), y.reshape(1, -1)).T.tolist()[0] 
             for y in dic_y.values()]
            ).T
#print(similarities)     

## adjust and rescale
labels = list(dic_y.keys())
for i in range(len(similarities_q)):
    ### assign randomly if there is no similarity
    if sum(similarities_q[i]) == 0:
       similarities_q[i] = [0]*len(labels)
       similarities_q[i][np.random.choice(range(len(labels)))] = 1
    ### rescale so they sum = 1
    similarities_q[i] = similarities_q[i] / sum(similarities_q[i])

## classify the label with highest similarity score
predicted_prob_q = similarities_q
predicted_q = [labels[np.argmax(pred)] for pred in predicted_prob_q]

print(predicted_q)

['PLACE']


# Template Matching

In [13]:
templates = pd.read_csv(predicted_q[0] + ".csv", sep='|', engine='python')
templates.head()

,Template,Cypher
0,Historical Info about <Place>,"MATCH (p) WHERE p.ns0__name = ""<Place>"" RETURN..."
1,Villages in Santorini,"MATCH (start{ns0__name:""Santorini""})-[r:ns0__c..."
2,What is Santorini also known as?,"MATCH (start{ns0__name:""Santorini""})-[r:ns1__A..."
3,Myths about <Place>,"MATCH (start{ns0__name:""<Place>""})-[r:ns1__Myt..."
4,How can I get around in <Place>?,"MATCH (start{ns0__name:""<Place>""})-[r:ns1__Get..."


In [15]:
#preprocessing and cleaning method 1

def utils_preprocess_text1(sentence):
  #Remove any < > references
  new_sentence = re.sub("<+[\w]*>", "", sentence)
    
  #Remove the word Santorini
  new_sentence = new_sentence.replace("Santorini", "")
  new_sentence = new_sentence.replace("santorini", "")
  
  return new_sentence

In [16]:
#preprocessing and cleaning method 2

'''
Preprocess a string.
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def utils_preprocess_text2(text, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()    
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [17]:
#Πειραμα 2.1.5
#Cleaning Templates with utils_preprocess_text1 + utils_preprocess_text2 AND Cleaning Question with utils_preprocess_text2

lst_stopwords = nltk.corpus.stopwords.words("english")
index=0
for sentence in templates["Template"]:
  sentence = utils_preprocess_text1(sentence)
  templates.at[index,'Template_Clean3'] = utils_preprocess_text2(sentence, flg_lemm=True)

  index=index+1

Question_simil_clean= utils_preprocess_text2(Question, flg_lemm=True)


In [18]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
template_embeddings = model.encode(templates["Template_Clean3"])
question_embeddings = model.encode(Question_simil_clean)

In [19]:
similarities2 = cosine_similarity(question_embeddings.reshape(1, -1), template_embeddings)

#for every question we calculate how similar it is to each template
print(similarities2)

for i in range(len(similarities2)):
    ### rescale so they sum = 1
    similarities2[i] = similarities2[i] / sum(similarities2[i])

## classify the label with highest similarity score
predicted_prob = similarities2
predicted = []
second =[]

for pred in predicted_prob:
  x = np.argmax(pred)
  predicted.append("T" + str(x))
  
  
  #getting the second biggest similarity
  temp = np.copy(pred)
  temp[x] = -1
  x2=np.argmax(temp)
  second.append("T" + str(x2))



[[0.12108386 0.3679518  0.08931907 0.07515872 0.2548027  0.09556055
  0.87027556 0.518022  ]]


In [20]:
#Cleaning Templates with utils_preprocess_text1 + utils_preprocess_text2 AND Cleaning Question with utils_preprocess_text2

index=0
for sentence in templates["Template"]:
  sentence = utils_preprocess_text1(sentence)
  templates.at[index,'Template_Clean3'] = utils_preprocess_text2(sentence, flg_lemm=True, lst_stopwords=lst_stopwords)

  index=index+1



Question_ling_clean = utils_preprocess_text2(Question, flg_lemm=True, lst_stopwords=lst_stopwords)
 


In [21]:
def Similarity_LinguisticApproach(question, templates):
  score = []
  for template in templates["Template_Clean3"]:
    sc=0
    for word in question.split():
      if word in template.split():
        sc=sc+1
    score.append(sc)

  return score    

In [22]:
scores = []
scores = Similarity_LinguisticApproach(Question_ling_clean, templates)

count=0
for i in scores:
  count=count+i

if count==0:
  scores.clear()


In [23]:
predicted_prob = scores
predicted2=[]

if predicted_prob:
  x = np.argmax(pred)
  predicted2.append("T" + str(x)) 
else:
  predicted2.append(None)


In [24]:
Final_predictions=[]

i=0
for p1, p2 in zip(predicted, predicted2):
  if p2:
    if p2==p1:
      final_Pred = p1
    elif p2==second[i]:
      final_Pred = second[i]
    else:
      final_Pred = p2
  else:
    final_Pred = p1
  
  i=i+1
  Final_predictions.append(final_Pred)


print("Final Predictions", Final_predictions)


Final Predictions ['T6']


#Answer Retrieval

In [25]:
index=0
for cyph_q in templates["Cypher"]:
  new_cyph_q = re.sub("<+[\w]*>", "#", cyph_q)
  templates.at[index,'Cypher_Clean'] = new_cyph_q
  index=index+1
  
templates["Cypher_Clean"].head()

0    MATCH (p) WHERE p.ns0__name = "#" RETURN p.ns1...
1    MATCH (start{ns0__name:"Santorini"})-[r:ns0__c...
2    MATCH (start{ns0__name:"Santorini"})-[r:ns1__A...
3    MATCH (start{ns0__name:"#"})-[r:ns1__Myths]->(...
4    MATCH (start{ns0__name:"#"})-[r:ns1__Get_Aroun...
Name: Cypher_Clean, dtype: object

In [26]:
!pip install py2neo
from py2neo import Graph

graph = Graph("neo4j+s://b6c3be65.databases.neo4j.io", auth=("neo4j", "santorinikg2023"))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
def answer_Retrieval(Final_predictions, Question, templates):
  for p in Final_predictions:
    #fetching the appropriate template
    template_index = int(p[1])
    cypher_query = templates["Cypher_Clean"].iloc[template_index]

    if cypher_query.count("#") > 0:
      cypher_query = QA_NER(Question, cypher_query)
    
    answer = graph.run(cypher_query).to_data_frame()
  
  return answer

In [28]:
def QA_NER(Question, cypher_query):
  NER = spacy.load("en_core_web_sm")
  
  #Truecasing the questions in order to apply NER method 
  n = truecase.get_true_case(Question)
  #Using NER to find the key entity that is required in the cypher query
  entities = NER(n)

  if cypher_query.count("#") == 1:
    for word in entities.ents:
      cypher_query = cypher_query.replace("#", str(word))
    
  elif cypher_query.count("#") == entities.ents.count:
    indexes = [x.start() for x in re.finditer('\#', cypher_query)]
        
    for i, word in zip(indexes, entities.ents):
      cypher_query = cypher_query[:i] + str(word) + cypher_query[i+1:] 
        
  return cypher_query

In [29]:
print("\nQuestion: ", Question)
answer = answer_Retrieval(Final_predictions, Question, templates)
print("Answer: ")
display(answer)



Question:  What are some places for families to visit
Answer: 


,Entity,Relation,Entity2_Name
0,Santorini,ns0__containsPlace,Kamari


In [30]:
def recommendation_retrieval(Final_predictions, previous_question, templates):
  for p in Final_predictions:
    #fetching the appropriate template
    template_index = int(p[1])
    cypher_query = templates["Cypher_Clean"].iloc[template_index]

    NER = spacy.load("en_core_web_sm")
    n = truecase.get_true_case(previous_question)
    #Using NER to find the key entity that is required in the cypher query
    entities = NER(n)

    if cypher_query.count("#") > 0:
      for word in entities.ents:
        cypher_query = cypher_query.replace("#", str(word))
      #print(cypher_query)    

    answer = graph.run(cypher_query).to_data_frame()
  
  return answer

# Recommendations

In [31]:
if Final_predictions != second:
  #fetching the appropriate template
  template_index = int(second[0][1])
  nl = templates["Template"].iloc[template_index]
  
  print("Would you like to know more about: ", nl)

  input1 = input("Type YES/NO to continue: ")
  if input1.upper() == "YES":
    try:
      answer2 = recommendation_retrieval(second, Question, templates)
      print("Answer: ")
      display(answer2)
    except:
      print("An exception occurred")
    
        



Would you like to know more about:  What are some romantic places?
Type YES/NO to continue: yes
Answer: 


,Entity,Relation,Entity2_Name
0,Santorini,ns0__containsPlace,Akrotiri
1,Santorini,ns0__containsPlace,Imerovigli
2,Santorini,ns0__containsPlace,Pyrgos
3,Santorini,ns0__containsPlace,Oia


In [32]:
print("Thank you! Bye Bye! :)")

Thank you! Bye Bye! :)
